In [7]:
from pathlib import Path
from llama_index import download_loader, VectorStoreIndex, load_index_from_storage
from llama_index.storage.storage_context import StorageContext

PDFReader = download_loader('PDFReader')
loader = PDFReader()

class PaperTitle:
    def __init__(self, name, about, file, key):
        self.name = name
        self.about = about
        self.file = file
        self.key = key
        
papers_titles = [
    PaperTitle('llama2', 'llama2 the name of large language models by metaAI', 'llama2.pdf', 'tools_for_llama2'),
    PaperTitle('LLM on Graph','Large Language Models using on Graph Structure','LLM_on_Graphs_A_Comprehensive_Survey.pdf', 'tools_for_GNN_or_recomendation_system')
]

paper_vector_index = {}
for paper in papers_titles:
    try:
        storage_context = StorageContext.from_defaults(persist_dir=f"./storage/cache/{paper.key}_vector")
        paper_vector_index[paper.key] = load_index_from_storage(storage_context)
    except:
        documents = loader.load_data(file=Path(f"./assets/{paper.file}"))
        vector_index = VectorStoreIndex.from_documents(documents)
        paper_vector_index[paper.key] = vector_index
        vector_index.storage_context.persist(persist_dir=f"./storage/cache/{paper.key}_vector")


In [8]:
paper_vector_index # 각 paper에 대해서 vector index가 생성됨

{'tools_for_llama2': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x1754fcd30>,
 'tools_for_GNN_or_recomendation_system': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x280d93b50>}

In [15]:
from llama_index.indices.postprocessor import KeywordNodePostprocessor
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index.tools import QueryEngineTool, ToolMetadata

node_processor = KeywordNodePostprocessor(
    exclude_keywords=["commecial"]
)

query_engine_tools = []
paper_vector_engines = {}

for paper in papers_titles:
    retriever = VectorIndexRetriever(
        index = paper_vector_index[paper.key],
        similarity_top_k = 3,
    )
    paper_vector_engines[paper.key] = RetrieverQueryEngine(
        retriever = retriever,
        node_postprocessors = [node_processor]
    )
    new_tool = QueryEngineTool(
        query_engine=paper_vector_engines[paper.key],
        metadata=ToolMetadata(
            name=f"{paper.key}_vector_tool",
            description=f"Useful for retrieving specific context from a paper {paper.name}. "
            f"Use When you need information related to {paper.about}"
            )
    )
    
    query_engine_tools.append(new_tool)

In [16]:
paper_vector_engines

{'tools_for_llama2': <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x2816f5f90>,
 'tools_for_GNN_or_recomendation_system': <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x2816f5de0>}

In [17]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

agent = OpenAIAgent.from_tools(query_engine_tools, llm=OpenAI(temperature=0, model='gpt-3.5-turbo-0613'), verbose=True)

In [18]:
response = agent.chat('Tell me about llama 2?')

Added user message to memory: Tell me about llama 2?
=== Calling Function ===
Calling function: tools_for_llama2_vector_tool with args: {
  "input": "What are the key features of llama 2?"
}
Got output: Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) that range in scale from 7 billion to 70 billion parameters. The fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. These models have demonstrated competitiveness with existing open-source chat models and have competency equivalent to some proprietary models on evaluation sets. Llama 2 uses an optimized transformer architecture and employs supervised fine-tuning (SFT) and reinforcement learning with human feedback (RLHF) to align with human preferences for helpfulness and safety. The models were trained between January 2023 and July 2023 and are intended for commercial and research use in English. The pretrained models can be adapted for a variety of natural language generation tas

In [19]:
print(response.response)

LLAMA 2 is a collection of pretrained and fine-tuned large language models (LLMs) developed by metaAI. Here are some key features of LLAMA 2:

1. Scale: LLAMA 2 consists of LLMs ranging in scale from 7 billion to 70 billion parameters. The larger models have more capacity to capture complex language patterns and generate high-quality text.

2. Fine-tuned for Dialogue: LLAMA 2 includes fine-tuned LLMs called LLAMA 2-Chat, which are specifically optimized for dialogue-based use cases. These models have been trained to generate contextually relevant and coherent responses in conversations.

3. Competitiveness: LLAMA 2-Chat models have demonstrated competitiveness with existing open-source chat models and are comparable to some proprietary models on evaluation sets. This means that LLAMA 2 can provide high-quality dialogue generation.

4. Optimized Transformer Architecture: LLAMA 2 utilizes an optimized transformer architecture, which is a state-of-the-art neural network architecture for n